## Търсене и извличане на информация. Приложение на дълбоко машинно обучение
> ### Стоян Михов  
> #### Зимен семестър 2021/2022

### Упражнение 2

За да работи програмата трябва да се свали корпус от публицистични текстове за Югоизточна Европа,
предоставен за некомерсиално ползване от Института за български език - БАН
Корпусът може да бъде свален от:
http://dcl.bas.bg/BulNC-registration/dl.php?dl=feeds/JOURNALISM.BG.zip
Архивът трябва да се разархивира в директорията, в която е програмата.

Преди да се стартира програмата на питон е необходимо да се активира съответното обкръжение с командата:
`conda activate tii`

##### Цел: Търсене на документи по думи

In [1]:
!ls "../JOURNALISM.BG"

C-MassMedia
JOURNALISM_DESCRIPTION.BG.pdf
JOURNALISM_DESCRIPTION.EN.pdf
JOURNALISM_METADATA.BG.txt
JOURNALISM_METADATA.EN.txt


In [43]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import sys
from numbers import Number
from collections import deque
from collections.abc import Set, Mapping


ZERO_DEPTH_BASES = (str, bytes, Number, range, bytearray)


def getsize(obj_0):
    """Recursively iterate to sum size of object & members."""
    _seen_ids = set()
    def inner(obj):
        obj_id = id(obj)
        if obj_id in _seen_ids:
            return 0
        _seen_ids.add(obj_id)
        size = sys.getsizeof(obj)
        if isinstance(obj, ZERO_DEPTH_BASES):
            pass # bypass remaining control flow and return
        elif isinstance(obj, (tuple, list, Set, deque)):
            size += sum(inner(i) for i in obj)
        elif isinstance(obj, Mapping) or hasattr(obj, 'items'):
            size += sum(inner(k) + inner(v) for k, v in getattr(obj, 'items')())
        # Check for custom object instances - may subclass above too
        if hasattr(obj, '__dict__'):
            size += inner(vars(obj))
        if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
            size += sum(inner(getattr(obj, s)) for s in obj.__slots__ if hasattr(obj, s))
        return size
    return inner(obj_0)

In [3]:
corpus_root = '../JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
fileNames = myCorpus.fileids()

In [4]:
myCorpus.words(fileNames[28])

['Гръцки', 'град', 'бе', 'избран', 'за', 'Европейска', ...]

In [5]:
len(fileNames)

35337

In [6]:
def printDocuments(myCorpus,fileNames,docIDs):
    for docID in docIDs:
        text = myCorpus.words(fileNames[docID])
        print('Document ID: '+str(docID))
        for word in text:
            print(word,end=' ')
        print('\n')

In [7]:
printDocuments(myCorpus,fileNames,[7,28])

Document ID: 7
Културен преглед 18 / 12 / 2002 Преглед на спортни , развлекателни и други културни новини от Балканите след 11 декември . ( Различни източници -- 11 / 12 / 02 - 18 / 12 / 02 ) Преглед на спортни , развлекателни и други културни новини от Балканите след 11 декември : Open Broadcast Network TV на Босна и Херцеговина и хърватската Нова ТВ се споразумяха да засилят двустранното сътрудничество през следващата година , като си разменят телевизионни програми . Мениджърите на двете ТВ станции се срещнаха в Сараево на 13 декември . *** На 12 декември в галерията " Препород " в Сараево беше открита нова изложба от босненския фотограф Мухамад Шишич . Изложбата включва снимки на планини от цял свят . *** Регионален семинар за възстановяване на народните занаяти се проведе в Скопие , Македония , от 12 до 15 декември . Участваха специалисти от Македония , България , Франция , Румъния и САЩ . *** В Музея на съвременното изкуство в Скопие , Македония , беше открита изложбата " Световна

In [ ]:
!du -shc "../JOURNALISM.BG"

###### Как да структурираме информацията?

In [8]:
class progressBar:
    def __init__(self ,barWidth = 80):
        self.barWidth = barWidth
        self.period = None

    def start(self, count):
        self.period = max(int(count / self.barWidth),1)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
        sys.stdout.write("\b" * (self.barWidth+1))

    def tick(self, item):
        if item % self.period == 0:
            sys.stdout.write("-")
            sys.stdout.flush()

    def stop(self):
        sys.stdout.write("]\n")

In [9]:
list(enumerate(['a','b','c']))

[(0, 'a'), (1, 'b'), (2, 'c')]

In [10]:
pb = progressBar()
pb.start(len(fileNames))
dictionary={}
for docID, fileName in enumerate(fileNames):
    pb.tick(docID)
    text = myCorpus.words(fileName)
    for token in text:
        if not token.isalpha(): continue
        term = token.lower()
        if term in dictionary:
            postings = dictionary[term]
            if postings[-1] < docID:
                postings.append(docID)
        else:
            dictionary[term] = [docID]
pb.stop()

[                                                                                ---------------------------------------------------------------------------------]


In [11]:
len(dictionary)

122739

In [12]:
p1 = dictionary['всички'];p1

[4,
 6,
 8,
 9,
 25,
 43,
 46,
 69,
 74,
 75,
 78,
 82,
 124,
 153,
 169,
 171,
 178,
 184,
 187,
 188,
 191,
 194,
 198,
 206,
 212,
 225,
 227,
 268,
 277,
 278,
 279,
 284,
 289,
 291,
 308,
 321,
 331,
 337,
 338,
 339,
 341,
 346,
 368,
 386,
 407,
 441,
 442,
 444,
 447,
 449,
 450,
 451,
 452,
 454,
 466,
 468,
 469,
 471,
 475,
 477,
 481,
 482,
 483,
 486,
 504,
 506,
 515,
 518,
 523,
 544,
 550,
 572,
 580,
 586,
 593,
 595,
 601,
 604,
 609,
 610,
 625,
 636,
 645,
 659,
 663,
 670,
 672,
 678,
 683,
 688,
 689,
 692,
 696,
 705,
 708,
 739,
 759,
 761,
 765,
 767,
 768,
 777,
 781,
 794,
 823,
 825,
 831,
 832,
 838,
 840,
 850,
 852,
 862,
 865,
 876,
 885,
 888,
 890,
 898,
 901,
 907,
 911,
 918,
 920,
 951,
 963,
 970,
 973,
 993,
 994,
 1008,
 1009,
 1020,
 1022,
 1053,
 1054,
 1059,
 1071,
 1075,
 1079,
 1085,
 1088,
 1090,
 1092,
 1093,
 1097,
 1099,
 1100,
 1102,
 1103,
 1104,
 1105,
 1112,
 1117,
 1122,
 1124,
 1126,
 1127,
 1132,
 1133,
 1134,
 1136,
 1142,
 1163

In [13]:
p2 = dictionary['хора']

##### Да реализираме операциите за сливане:

In [14]:
def intersect(p1,p2):
    i1=0
    i2=0
    answer=[]
    while i1<len(p1) and i2<len(p2):
        if p1[i1] == p2[i2]:
            answer.append(p1[i1])
            i1 += 1
            i2 += 1
        elif p1[i1] < p2[i2]:
            i1 += 1
        else:
            i2 += 1
    return answer

In [15]:
p1

[4,
 6,
 8,
 9,
 25,
 43,
 46,
 69,
 74,
 75,
 78,
 82,
 124,
 153,
 169,
 171,
 178,
 184,
 187,
 188,
 191,
 194,
 198,
 206,
 212,
 225,
 227,
 268,
 277,
 278,
 279,
 284,
 289,
 291,
 308,
 321,
 331,
 337,
 338,
 339,
 341,
 346,
 368,
 386,
 407,
 441,
 442,
 444,
 447,
 449,
 450,
 451,
 452,
 454,
 466,
 468,
 469,
 471,
 475,
 477,
 481,
 482,
 483,
 486,
 504,
 506,
 515,
 518,
 523,
 544,
 550,
 572,
 580,
 586,
 593,
 595,
 601,
 604,
 609,
 610,
 625,
 636,
 645,
 659,
 663,
 670,
 672,
 678,
 683,
 688,
 689,
 692,
 696,
 705,
 708,
 739,
 759,
 761,
 765,
 767,
 768,
 777,
 781,
 794,
 823,
 825,
 831,
 832,
 838,
 840,
 850,
 852,
 862,
 865,
 876,
 885,
 888,
 890,
 898,
 901,
 907,
 911,
 918,
 920,
 951,
 963,
 970,
 973,
 993,
 994,
 1008,
 1009,
 1020,
 1022,
 1053,
 1054,
 1059,
 1071,
 1075,
 1079,
 1085,
 1088,
 1090,
 1092,
 1093,
 1097,
 1099,
 1100,
 1102,
 1103,
 1104,
 1105,
 1112,
 1117,
 1122,
 1124,
 1126,
 1127,
 1132,
 1133,
 1134,
 1136,
 1142,
 1163

In [16]:
p2

[1,
 39,
 43,
 52,
 62,
 64,
 65,
 69,
 81,
 96,
 108,
 120,
 129,
 135,
 137,
 160,
 178,
 183,
 192,
 210,
 231,
 233,
 235,
 238,
 248,
 265,
 315,
 333,
 339,
 347,
 358,
 386,
 393,
 402,
 417,
 420,
 430,
 442,
 444,
 450,
 452,
 453,
 455,
 459,
 466,
 467,
 468,
 478,
 479,
 481,
 482,
 483,
 525,
 544,
 580,
 593,
 606,
 639,
 663,
 672,
 679,
 761,
 781,
 797,
 811,
 823,
 826,
 831,
 838,
 850,
 864,
 872,
 901,
 907,
 963,
 966,
 993,
 1023,
 1025,
 1054,
 1075,
 1083,
 1096,
 1101,
 1104,
 1118,
 1121,
 1122,
 1125,
 1130,
 1134,
 1141,
 1182,
 1221,
 1231,
 1298,
 1340,
 1426,
 1442,
 1453,
 1534,
 1576,
 1617,
 1622,
 1623,
 1628,
 1629,
 1637,
 1649,
 1653,
 1720,
 1917,
 1935,
 1939,
 2046,
 2081,
 2082,
 2086,
 2090,
 2126,
 2133,
 2216,
 2224,
 2253,
 2255,
 2296,
 2309,
 2318,
 2359,
 2365,
 2372,
 2427,
 2460,
 2473,
 2491,
 2521,
 2523,
 2576,
 2593,
 2612,
 2614,
 2642,
 2664,
 2719,
 2751,
 2785,
 2814,
 2857,
 2942,
 2963,
 2988,
 3003,
 3005,
 3028,
 3055,
 31

In [17]:
intersect(p1,p2)

[43,
 69,
 178,
 339,
 386,
 442,
 444,
 450,
 452,
 466,
 468,
 481,
 482,
 483,
 544,
 580,
 593,
 663,
 672,
 761,
 781,
 823,
 831,
 838,
 850,
 901,
 907,
 963,
 993,
 1054,
 1075,
 1104,
 1122,
 1134,
 1221,
 1298,
 1340,
 1629,
 1720,
 2046,
 2086,
 2365,
 2372,
 2473,
 2523,
 2612,
 2664,
 2785,
 2857,
 2942,
 3199,
 3239,
 3339,
 3366,
 3379,
 3453,
 3605,
 3651,
 3795,
 3971,
 4463,
 4527,
 4920,
 5107,
 5161,
 5175,
 5436,
 5745,
 5955,
 6014,
 6040,
 6081,
 6206,
 6229,
 6459,
 6697,
 6780,
 6981,
 7109,
 7223,
 7257,
 7338,
 7374,
 7550,
 7612,
 7649,
 7686,
 7742,
 7780,
 7825,
 7904,
 7972,
 7980,
 7981,
 7984,
 7987,
 7995,
 8172,
 8306,
 8336,
 8340,
 8349,
 8350,
 8355,
 8377,
 8393,
 8397,
 8398,
 8400,
 8404,
 8407,
 8409,
 8417,
 8418,
 8422,
 8423,
 8429,
 8431,
 8432,
 8436,
 8440,
 8442,
 8447,
 8450,
 8451,
 8453,
 8457,
 8459,
 8460,
 8464,
 8469,
 8479,
 8483,
 8486,
 8489,
 8565,
 8587,
 8618,
 8664,
 8741,
 8759,
 9035,
 9202,
 9514,
 9631,
 9668,
 9774,
 9

##### Сечение на повече от един списък?

In [18]:
def intersectLists(andLists):
    if not andLists: return []
    postingLists = sorted(andLists, key = lambda x : len(x))
    answer = postingLists[0]
    for i in range(1,len(postingLists)):
        answer = intersect(answer,postingLists[i])
    return answer

In [19]:
def andSearch(dictionary, termList):
    dicItems = [ dictionary[term] for term in termList if term in dictionary ]
    return intersectLists(dicItems)

In [20]:
andSearch(dictionary,['бизнес'])

[1,
 5,
 14,
 91,
 117,
 374,
 430,
 438,
 455,
 474,
 475,
 484,
 487,
 489,
 491,
 493,
 495,
 500,
 502,
 504,
 505,
 507,
 509,
 511,
 514,
 517,
 518,
 520,
 521,
 524,
 526,
 534,
 536,
 539,
 544,
 547,
 548,
 551,
 554,
 555,
 556,
 558,
 562,
 563,
 564,
 565,
 567,
 568,
 569,
 570,
 573,
 576,
 579,
 582,
 584,
 592,
 593,
 639,
 671,
 705,
 718,
 737,
 742,
 759,
 762,
 765,
 787,
 825,
 826,
 827,
 833,
 836,
 838,
 845,
 850,
 851,
 857,
 860,
 861,
 863,
 865,
 869,
 876,
 877,
 889,
 905,
 907,
 911,
 920,
 922,
 923,
 924,
 934,
 945,
 950,
 957,
 962,
 971,
 978,
 980,
 987,
 993,
 996,
 1002,
 1015,
 1019,
 1020,
 1040,
 1043,
 1053,
 1054,
 1066,
 1068,
 1081,
 1082,
 1084,
 1085,
 1086,
 1087,
 1088,
 1090,
 1093,
 1097,
 1099,
 1101,
 1103,
 1106,
 1107,
 1108,
 1111,
 1115,
 1119,
 1120,
 1122,
 1124,
 1126,
 1128,
 1222,
 1225,
 1249,
 1300,
 1315,
 1534,
 1536,
 1544,
 1573,
 1621,
 1623,
 1742,
 1791,
 1808,
 1952,
 2026,
 2099,
 2273,
 2428,
 2464,
 2622,
 27

In [21]:
def union(p1,p2):
    i1 = 0
    i2 = 0
    answer = []
    while i1 < len(p1) or i2 < len(p2):
        if i2 == len(p2) or (i1 < len(p1) and p1[i1] < p2[i2]):
            answer.append(p1[i1])
            i1 += 1
        elif i1 == len(p1) or (i2 < len(p2) and p2[i2] < p1[i1]):
            answer.append(p2[i2])
            i2 += 1
        else:
            answer.append(p1[i1])
            i1 += 1
            i2 += 1
    return answer

##### Цел: търсене на документи по шаблони

In [22]:
class Trie:
    def __init__(self):
        self.stateTransitions = [{}]
        self.finalStates = set([])

    def traverse(self,word,initialState = 0):
        s = initialState # current state
        i = 0 # current position in word
        while i < len(word) and word[i] in self.stateTransitions[s]:
            s = self.stateTransitions[s][word[i]]
            i += 1
        return s, i

    def isFinal(self, s):
        return s in self.finalStates

    def inTrie(self, word):
        s, i = self.traverse(word)
        return i == len(word) and self.isFinal(s)

    def addWord(self,word):
        s, i = self.traverse(word) # s = 8; i->ч
        while i < len(word):
            newState = len(self.stateTransitions) # 14
            self.stateTransitions.append({})
            st = self.stateTransitions[s]
            st[word[i]] = newState
            s = newState
            i += 1
        self.finalStates.add(s)

    def getWordsFromState(self,s):
        answer = []
        if s in self.finalStates:
            answer.append('')
        st = self.stateTransitions[s]
        for (char,nextstate) in st.items():
            answer += [ char + word for word in self.getWordsFromState(nextstate) ]
        return answer

    def getWordsWithPrefix(self,prefix):
        s,i = self.traverse(prefix)
        if i != len(prefix):
            return []
        else:
            return [ prefix + word for word in self.getWordsFromState(s) ]

In [23]:
dictionaryTrie=Trie()

In [24]:
len(dictionary)

122739

In [37]:
len(dictionaryTrie.stateTransitions)

293480

In [44]:
getsize(dictionary)/1024/1024

62.55008888244629

In [46]:
getsize(dictionaryTrie)/1024/1024

102.53844356536865

In [35]:
for term in list(dictionary):
    dictionaryTrie.addWord(term)

In [39]:
def andSearchWithWildCards(dictionary,dictionaryTrie,patternList):
    items = []
    for pattern in patternList:
        orList = []
        for term in dictionaryTrie.getWordsWithPrefix(pattern):
            orList = union(orList,dictionary[term])
        items.append(orList)
    return intersectLists(items)

In [40]:
andSearchWithWildCards(dictionary, dictionaryTrie, ['култур','обмен','теат'])

[128, 138, 139, 140, 176, 179, 246, 265, 276, 337, 442, 19885, 31590, 32403]

In [41]:
printDocuments(myCorpus,fileNames,[128, 138, 139, 140, 176, 179, 246, 265, 276, 337, 442, 19885, 31590, 32403])

Document ID: 128
Културен преглед 23 / 02 / 2005 Преглед на спортните , развлекателните и други културни новини от Балканите след 16 февруари ( Различни източници – 16 / 02 / 05 – 23 / 02 / 05 ) Албанският национален театър за опера и балет гастролира за пръв път в Македония на 18 февруари с операта " Любовно биле " от Гаетано Доницети в Скопие . Базираната в Македония фондация " Балкани без граници " финансира тази проява и ще спонсорира предстоящ концерт в Тирана на Македонския оперен театър . [ Toмислав Георгиев ] Три изложби се представят понастоящем в Националната галерия по изкуствата в Тирана . Първата , " Синапси ", предлага творби на млади албански и италиански художници . Отделна изложба представя младия албански творец Елира Суло , а фотоизложбата с произведения на германския фотограф Регина Шмекен ще бъде отворена за публиката до края на февруари . *** Хърватия и Йордания подписаха споразумение за сътрудничество в областта на културата по време на визитата на премиера Иво С

Културна хроника 14 / 12 / 2005 Преглед на спортните , развлекателни и културни новини от Югоизточна Европа след 7 декември . ( Различни източници – 07 / 12 / 05 – 14 / 12 / 05 ) Румънският президент Траян Бъсеску ( вдясно ) изпи чаша с вино с молдовския президент Владимир Воронин по време на церемонията по откриване на Молдовския фестивал на виното в Букурещ в събота ( 10 декември ). [ АФП ] Филмът на немския режисьор Хайко Хан " Преди да си тръгна " спечели наградата за най - добър филм на 3 - ото издание на Международния фестивал за късометражни филми в Тирана , който се проведе от 5 декември до 11 декември в столицата на Aлбания . Тази година наградата на публиката бе връчена на албанския режисьор Роберт Будина за филма му " Кокичета ". *** Третата Международна художествена колония бе открита в Яхорина , Босна и Херцеговина , на 10 декември . Участват около 15 босненски и чуждестранни художници . След това изложби на творби , създадени по време на тазгодишното и миналите издания на

Култура и спорт : Проведе се 25 - тият Атински класически маратон 07 / 11 / 2007 Атинският класически маратон се проведе в Гърция . Още новини от седмицата : Кинофестивалът " Киномания " започна в София и се провежда конкурс за най - красивата дума в света . Участник в Атинския класически маратон 2007 в неделя ( 4 ноември ) [ Гети Имиджис ] Бенджамин Кипротич Корин от Кения и рускинята Светлана Пономаренко спечелиха 25 - тия Атински класически маратон в гръцката столица в неделя ( 4 ноември . Маратонът следва маршрута на състезанието от древността , като започва от крайбрежния град Маратон и свършва на стадион " Панатинайкон " в Атина . *** Tурската дума yakamoz , която означава " отражение на луната във водата ", спечели състезанието за най - красива дума в света . Събитието бе организирано от германското електронно списание KulturAustausch ( Културен обмен ). *** Семинар по дизайн , организиран от швейцарската културна програма в Maкедония , започна в понеделник ( 5 ноември ) в Скопи

Македонски детски театър носи радост 04 / 08 / 2010 Културата е сила , която може да преодолява предразсъдъците и да прекосява границите , казва режисьорът Любомир Чадиковски . От Марина Стояновска за Southeast European Times от Скопие -- 04 / 08 / 10 Режисьорът Любомир Чадиковски работи в театъра от неговото създаване преди повече от 20 години . [ Томислав Георгиев / SETimes ] В продължение на повече от 20 години младите хора масово посещават наградените постановки на македонския Театър за деца и юноши , навлизайки с света на фантазиите , емоциите и фолклора . С репертоар , включващ пиеси като " Красавицата и звяра ", " Малкият принц ", " Малката Русалка " и " Крали Марко " театърът продължава да омайва своята публика . Кореспондентът на SETimes Марина Стояновска беседва с режисьора Любомир Чадиковски , който работи в театъра от неговото създаване . SETimes : Какво е чувството да чествате 20 - годишнината на театъра ? И как поддържате интереса сред най - трудната публика , децата ? Лю